In [8]:
import pyspark
import pandas as pd
import numpy as np
spark = pyspark.sql.SparkSession.builder.getOrCreate()

## 1.Create a spark data frame that contains your favorite programming languages.

- The name of the column should be language
- View the schema of the dataframe
- Output the shape of the dataframe
- Show the first 5 records in the dataframe

In [13]:
pandas_dataframe = pd.DataFrame(
    {'languages':['python','c++','c','java','javascript','typescript','ruby','r','scala']}
)
pandas_dataframe

,languages
0,python
1,c++
2,c
3,java
4,javascript
5,typescript
6,ruby
7,r
8,scala


In [14]:
df.printSchema()

root
 |-- language: string (nullable = true)



In [15]:
df.show()

+-----------+
|   language|
+-----------+
|          r|
|     python|
|        sql|
|      julia|
|  pig latin|
|lorem ipsum|
+-----------+



In [16]:
df = spark.createDataFrame(pandas_dataframe)
df.show(5)

+----------+
| languages|
+----------+
|    python|
|       c++|
|         c|
|      java|
|javascript|
+----------+
only showing top 5 rows



In [17]:
print("DataFrame shape: ", df.count(), " x ", len(df.columns))


DataFrame shape:  9  x  1


## 2. Load the mpg dataset as a spark dataframe.

- Create 1 column of output that contains a message like the one below:


`The 1999 audi a4 has a 4 cylinder engine`
- For each vehicle.

- Transform the trans column so that it only contains either manual or auto.

In [18]:
from pydataset import data

mpg = spark.createDataFrame(data("mpg"))
mpg.show(5)

+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
|        audi|   a4|  2.8|1999|  6|  auto(l5)|  f| 16| 26|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 5 rows



In [19]:
mpg.select(
    concat(
        lit('The '), 
        mpg.year, 
        lit(' '), 
        mpg.manufacturer,
        lit(' '), 
        mpg.model, 
        lit(' has a '), 
        mpg.cyl, 
        lit(' cylinder engine'))
).show(20,False)

+-----------------------------------------------------------------------------+
|concat(The , year,  , manufacturer,  , model,  has a , cyl,  cylinder engine)|
+-----------------------------------------------------------------------------+
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 2008 audi a4 has a 4 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
|The 1999 audi a4 has a 6 cylinder engine                                     |
|The 2008 audi a4 has a 6 cylinder engine                                     |
|The 1999 audi a4 quattro has a 4 cylinder engine                             |
|The 1999 audi a4 quattro has a 4 cylinder engine                             |
|The 2008 audi a4 quattro has a 4 cylind

In [20]:
### Transform each column so that it only contains either manual or atuo

In [21]:
from pyspark.sql.functions import regexp_extract, regexp_replace
from pyspark.sql.functions import when

In [24]:
mpg.createOrReplaceTempView("mpg")

In [25]:
mpg.show(4)


+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|manufacturer|model|displ|year|cyl|     trans|drv|cty|hwy| fl|  class|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
|        audi|   a4|  1.8|1999|  4|  auto(l5)|  f| 18| 29|  p|compact|
|        audi|   a4|  1.8|1999|  4|manual(m5)|  f| 21| 29|  p|compact|
|        audi|   a4|  2.0|2008|  4|manual(m6)|  f| 20| 31|  p|compact|
|        audi|   a4|  2.0|2008|  4|  auto(av)|  f| 21| 30|  p|compact|
+------------+-----+-----+----+---+----------+---+---+---+---+-------+
only showing top 4 rows



In [26]:
mpg.select(
    'trans',
    regexp_extract("trans", r"^(\w+)\(", 1).alias("regexp_extract"),
    regexp_replace("trans", r"\(.+$", "").alias("regexp_replace"),
    when(
        mpg.trans.like("auto%"), "auto"
    ).otherwise("manual").alias("when + like")
).show()

+----------+--------------+--------------+-----------+
|     trans|regexp_extract|regexp_replace|when + like|
+----------+--------------+--------------+-----------+
|  auto(l5)|          auto|          auto|       auto|
|manual(m5)|        manual|        manual|     manual|
|manual(m6)|        manual|        manual|     manual|
|  auto(av)|          auto|          auto|       auto|
|  auto(l5)|          auto|          auto|       auto|
|manual(m5)|        manual|        manual|     manual|
|  auto(av)|          auto|          auto|       auto|
|manual(m5)|        manual|        manual|     manual|
|  auto(l5)|          auto|          auto|       auto|
|manual(m6)|        manual|        manual|     manual|
|  auto(s6)|          auto|          auto|       auto|
|  auto(l5)|          auto|          auto|       auto|
|manual(m5)|        manual|        manual|     manual|
|  auto(s6)|          auto|          auto|       auto|
|manual(m6)|        manual|        manual|     manual|
|  auto(l5

## 3.Load the tips dataset as a spark dataframe.

- What percentage of observations are smokers?
- Create a column that contains the tip percentage
- Calculate the average tip percentage for each combination of sex and smoker.

In [29]:
## tips
tips = spark.createDataFrame(data("tips"))
tips.show(5)


+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows



In [30]:
tips.groupBy("smoker").count().show()

+------+-----+
|smoker|count|
+------+-----+
|    No|  151|
|   Yes|   93|
+------+-----+



In [31]:
tips.groupBy("smoker").count().withColumn("percent", concat(round((
col("count")/tips.count()*100),0).cast("int"), lit("%")),).show()

+------+-----+-------+
|smoker|count|percent|
+------+-----+-------+
|    No|  151|    62%|
|   Yes|   93|    38%|
+------+-----+-------+



In [32]:
tips.withColumn("tip_percentage", col('tip') / col('total_bill')).show()


+----------+----+------+------+---+------+----+-------------------+
|total_bill| tip|   sex|smoker|day|  time|size|     tip_percentage|
+----------+----+------+------+---+------+----+-------------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|0.05944673337257211|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|0.16054158607350097|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|0.16658733936220846|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2| 0.1397804054054054|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|0.14680764538430255|
|     25.29|4.71|  Male|    No|Sun|Dinner|   4|0.18623962040332148|
|      8.77| 2.0|  Male|    No|Sun|Dinner|   2|0.22805017103762829|
|     26.88|3.12|  Male|    No|Sun|Dinner|   4|0.11607142857142858|
|     15.04|1.96|  Male|    No|Sun|Dinner|   2|0.13031914893617022|
|     14.78|3.23|  Male|    No|Sun|Dinner|   2| 0.2185385656292287|
|     10.27|1.71|  Male|    No|Sun|Dinner|   2| 0.1665043816942551|
|     35.26| 5.0|Female|    No|Sun|Dinner|   4|0

In [34]:
tips.withColumn("tip_percentage", col('tip') / col('total_bill'))\
.groupby("sex")\
.pivot("smoker")\
.agg(round(mean("tip_percentage"), 4))\
.show()

+------+------+------+
|   sex|    No|   Yes|
+------+------+------+
|Female|0.1569|0.1822|
|  Male|0.1607|0.1528|
+------+------+------+



In [35]:
(
    tips.withColumn("tip_percentage", col('tip') / col('total_bill'))
    .groupby("sex")
    .pivot("smoker") # make a pivot table
    .agg(round(mean("tip_percentage"), 4))
    .show()
)

+------+------+------+
|   sex|    No|   Yes|
+------+------+------+
|Female|0.1569|0.1822|
|  Male|0.1607|0.1528|
+------+------+------+



## 4. Use the seattle weather dataset referenced in the lesson to answer the questions below.

- Convert the temperatures to fahrenheit.
- Which month has the most rain, on average?
- Which year was the windiest?
- What is the most frequent type of weather in January?
- What is the average high and low temperature on sunny days in July in 2013 and 2014?
- What percentage of days were rainy in q3 of 2015?
- For each year, find what percentage of days it rained (had non-zero precipitation).

In [67]:
from vega_datasets import data
weather = data.seattle_weather().assign(date=lambda df: df.date.astype(str))
weather = spark.createDataFrame(weather)
weather.show(5)

+----------+-------------+--------+--------+----+-------+
|      date|precipitation|temp_max|temp_min|wind|weather|
+----------+-------------+--------+--------+----+-------+
|2012-01-01|          0.0|    12.8|     5.0| 4.7|drizzle|
|2012-01-02|         10.9|    10.6|     2.8| 4.5|   rain|
|2012-01-03|          0.8|    11.7|     7.2| 2.3|   rain|
|2012-01-04|         20.3|    12.2|     5.6| 4.7|   rain|
|2012-01-05|          1.3|     8.9|     2.8| 6.1|   rain|
+----------+-------------+--------+--------+----+-------+
only showing top 5 rows



In [69]:
# pandas equivalent -- df.temp_max = df.temp_max * 9 / 5 + 32

weather = weather.withColumn(
    "temp_max", (col("temp_max") * 9 / 5 + 32)
).withColumn("temp_min", (col("temp_min") * 9 / 5 + 32))

In [70]:
weather.show(5)

+----------+-------------+------------------+--------+----+-------+
|      date|precipitation|          temp_max|temp_min|wind|weather|
+----------+-------------+------------------+--------+----+-------+
|2012-01-01|          0.0|           131.072|   105.8| 4.7|drizzle|
|2012-01-02|         10.9|123.94399999999999|  98.672| 4.5|   rain|
|2012-01-03|          0.8|127.50800000000001| 112.928| 2.3|   rain|
|2012-01-04|         20.3|           129.128| 107.744| 4.7|   rain|
|2012-01-05|          1.3|           118.436|  98.672| 6.1|   rain|
+----------+-------------+------------------+--------+----+-------+
only showing top 5 rows



In [60]:
## month most rain

In [71]:
row = (
    weather.withColumn("month", month("date"))
    .withColumn("year", year("date"))
    .groupBy("month", "year")
    .agg(sum("precipitation").alias("total_monthly_precipitation"))
    .groupBy("month")
    .agg(mean("total_monthly_precipitation").alias("avg_monthly_rain"))
    .sort(col("avg_monthly_rain").desc())
    .first()
)
row

Row(month=11, avg_monthly_rain=160.625)

In [72]:
row.avg_monthly_rain


160.625

In [ ]:
# year windiest

In [74]:
(
    weather.withColumn("year", year("date"))
    .groupBy("year")
    .agg(sum("wind").alias("total_winds"))
    .sort(col("total_winds").desc())
    .head(5)
)

[Row(year=2012, total_winds=1244.7),
 Row(year=2014, total_winds=1236.5000000000007),
 Row(year=2015, total_winds=1153.3000000000002),
 Row(year=2013, total_winds=1100.8000000000006)]

In [75]:
(
    weather.withColumn("month", month("date"))
    .filter(col("month") == 1)
    .groupBy("weather")
    .count()
    .sort(col("count").desc())
    .show()
)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



In [ ]:
## january weather

In [76]:
(
    weather.withColumn("month", month("date"))
    .filter(col("month") == 1)
    .groupBy("weather")
    .count()
    .sort(col("count").desc())
    .show()
)

+-------+-----+
|weather|count|
+-------+-----+
|    fog|   38|
|   rain|   35|
|    sun|   33|
|drizzle|   10|
|   snow|    8|
+-------+-----+



In [ ]:
## July

In [77]:
(
    weather.filter(month("date") == 7)
    .filter(year("date") > 2012)
    .filter(year("date") < 2015)
    .filter(col("weather") == lit("sun"))
    .agg(
        avg("temp_max").alias("average_high_temp"),
        avg("temp_min").alias("average_low_temp"),
    )
    .show()
)

+------------------+-----------------+
| average_high_temp| average_low_temp|
+------------------+-----------------+
|176.52546153846157|135.5519230769231|
+------------------+-----------------+



In [ ]:
# q3 2015

In [79]:
rainy_q3_2015 = (
    weather.filter(year("date") == 2015)
    .filter(quarter("date") == 3)
    .select(when(col("weather") == "rain", 1).otherwise(0).alias("rain"))
    .agg(mean("rain"))
)

In [81]:
rainy_q3_2015.show()

+--------------------+
|           avg(rain)|
+--------------------+
|0.021739130434782608|
+--------------------+



In [82]:
# For each year, find what percentage of days it rained (had non-zero precipitation).

In [84]:
# measure a rainy day by precipitation > 0
(
    weather.withColumn("year", year("date"))
    .select(when(col("precipitation") > 0, 1).otherwise(0).alias("rain"), "year")
    .groupby("year")
    .agg(mean("rain"))
    .show()
)

+----+-------------------+
|year|          avg(rain)|
+----+-------------------+
|2012|0.48360655737704916|
|2013|0.41643835616438357|
|2014|  0.410958904109589|
|2015|0.39452054794520547|
+----+-------------------+

